# 1. Gerekli Kütüphaneler

In [34]:
from dotenv import load_dotenv
import os
import openai
from groq import Groq
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 2. Ortam Değişkenlerini Yükle

In [35]:
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
print("GROQ:", groq_api_key[:5], "...")

GROQ: gsk_p ...


# 3. Hikayeler ve Kullanıcı Sorusu

In [36]:
stories = [
    "Ormanda kaybolan bir grup maceracı vardı.",
    "Küçük bir köyde herkes birbirine yardım ederdi.",
    "Bir gün deniz kenarında ilginç bir taş bulundu.",
    "Şehirde büyük bir festival düzenlendi.",
    "Yaşlı adam her sabah parkta yürüyüş yapardı.",
    "Çocuklar okul bahçesinde oyun oynuyordu.",
    "Bir köpek sahibini bulmak için uzun bir yolculuğa çıktı.",
    "Kütüphanede eski bir kitap keşfedildi.",
    "Dağ köyünde bir gece fırtına çıktı.",
    "Genç kadın yeni bir işe başladı."
]

In [37]:
user_question = "Ormanda ne oldu?"

# 4. Embedding Modelini Yükle

In [38]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# # model = SentenceTransformer("all-MiniLM-L6-v2")

C:\Users\HP\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Embedding -stories

In [39]:
story_embeddings = model.encode(stories)

### Embedding -question

In [40]:
question_embedding = model.encode(user_question)
# [model.encode(story) for story in stories]

In [53]:
for i, emb in enumerate(story_embeddings):
    print(f"Cümle: {stories[i]}")
    print(f"Embedding vektörü (ilk 5 değer): {emb[:5]}")
    print(f"Vektör boyutu: {len(emb)}")
    print()


Cümle: Ormanda kaybolan bir grup maceracı vardı.
Embedding vektörü (ilk 5 değer): [-0.00706573  0.05989986 -0.04916271 -0.01362983 -0.11429298]
Vektör boyutu: 384

Cümle: Küçük bir köyde herkes birbirine yardım ederdi.
Embedding vektörü (ilk 5 değer): [-0.03265146  0.06993041 -0.0396608  -0.00230425 -0.11561898]
Vektör boyutu: 384

Cümle: Bir gün deniz kenarında ilginç bir taş bulundu.
Embedding vektörü (ilk 5 değer): [-0.06433202  0.07662983 -0.0634205  -0.030231   -0.11734749]
Vektör boyutu: 384

Cümle: Şehirde büyük bir festival düzenlendi.
Embedding vektörü (ilk 5 değer): [-0.02297475  0.11371834  0.03535749 -0.0610215  -0.1018513 ]
Vektör boyutu: 384

Cümle: Yaşlı adam her sabah parkta yürüyüş yapardı.
Embedding vektörü (ilk 5 değer): [ 0.01251965  0.08531465  0.04876462 -0.02009429 -0.08396397]
Vektör boyutu: 384

Cümle: Çocuklar okul bahçesinde oyun oynuyordu.
Embedding vektörü (ilk 5 değer): [-0.02719678  0.09604759  0.00199794  0.04414297 -0.08131072]
Vektör boyutu: 384

Cümle

In [51]:
sentence = "Transformers mimarisi çok güçlüdür."
embedding = model.encode(sentence)
print(embedding.shape)

(384,)


# 7. Cosine Benzerliğini Hesapla

In [ ]:
similarities = cosine_similarity([question_embedding], story_embeddings)
best_match_idx = np.argmax(similarities)
best_story = stories[best_match_idx]

# 8. Groq ile Yanıt Oluştur

In [45]:
client = Groq(api_key = groq_api_key)

prompt = f""" 
Kullanıcı sorusu: {user_question},
İlgili Hikaye: {best_story}

Bu bilgileri kullanarak kullanıcı sorusunu cevapla.
"""

In [46]:
response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Sen bir hikaye yardımcısısın. Verilen hikayeyi kullanarak soruları cevapla."},
        {"role": "user", "content": prompt}
    ],
    model="llama3-70b-8192",
    temperature = 0.7,
    max_tokens = 500
)

In [47]:
print("Cevap:", response.choices[0].message.content)
print("\nİlgili Hikaye:", best_story)

Cevap: Üzgünüm, hikayede ormanda bir şey olmadı. Hikayede çocuklar okul bahçesinde oyun oynuyorlardı, ormanda hiçbir şey olmadı.

İlgili Hikaye: Çocuklar okul bahçesinde oyun oynuyordu.
